In [1]:
!git clone https://github.com/mohamedamine99/Object-tracking-and-counting-using-YOLOV8

Cloning into 'Object-tracking-and-counting-using-YOLOV8'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 90 (delta 0), reused 14 (delta 0), pack-reused 75
Unpacking objects: 100% (90/90), 244.01 MiB | 8.81 MiB/s, done.
Updating files: 100% (49/49), done.


In [2]:
import numpy as np
import random
import os
import cv2

import imageio
import time
import matplotlib.pyplot as plt



In [3]:
%pip install ultralytics
import ultralytics
ultralytics.checks()
from ultralytics import YOLO


Ultralytics YOLOv8.0.58 🚀 Python-3.9.16 torch-1.13.1+cu116 CPU
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.1/107.7 GB disk)


In [4]:
class YOLOv8_ObjectDetector:

    def __init__(self, model_file = 'yolov8n.pt', labels= None, classes = None, conf = 0.25, iou = 0.45 ):

        self.classes = classes
        self.conf = conf
        self.iou = iou

        self.model = YOLO(model_file)
        self.model_name = model_file.split('.')[0]

        if labels == None:
            self.labels = self.model.names


    def predict_img(self, img, verbose = True):
        results = self.model(img, classes = self.classes, conf = self.conf, iou = self.iou , verbose = verbose)
        self.orig_img = img
        self.results = results[0]
        return results[0]


    def default_display(self,show_conf=True, line_width=None, 
                        font_size=None, font='Arial.ttf', pil = False, example='abc'):
        
        return self.results.plot(show_conf , line_width , font_size , font, pil , example )
        

    def custom_display(self, colors, show_cls = True, show_conf = True ):

        img = self.orig_img
        # calculate the bounding box thickness based on the image width and height
        bbx_thickness = (img.shape[0] + img.shape[1]) // 450

        for box in self.results.boxes:
            textString = ""

            # Extract object class and confidence score
            score = box.conf.item() * 100
            class_id = int(box.cls.item())

            x1 , y1 , x2, y2 = np.squeeze(box.xyxy.numpy()).astype(int)

            # Calculate font scale based on object size
            if show_cls:
                textString += f"{self.labels[class_id]}"

            if show_conf:
                textString += f"{score:,.2f}%"
            #textString = f"{self.labels[class_id]}"  f" {score:,.2f}%"
            font = cv2.FONT_HERSHEY_COMPLEX
            fontScale = (((x2 - x1) / img.shape[0]) + ((y2 - y1) / img.shape[1])) / 2 * 1.5
            fontThickness = 1
            textSize, baseline = cv2.getTextSize(textString, font, fontScale, fontThickness)


            # Draw bounding box, a centroid and label on the image
            img = cv2.rectangle(img, (x1,y1), (x2,y2), colors[class_id], bbx_thickness)
            center_coordinates = ((x1 + x2)//2, (y1 + y2) // 2)

            img =  cv2.circle(img, center_coordinates, 5 , (0,0,255), -1)

            if textString != "":
                if (y1 < textSize[1]):
                    y1 = y1 + textSize[1]
                    
                else:
                    y1 -= 2
            
                img = cv2.rectangle(img, (x1, y1), (x1 + textSize[0] , y1 -  textSize[1]), colors[class_id], cv2.FILLED)
                img = cv2.putText(img, textString , 
                    (x1, y1), font, 
                fontScale,  (0, 0, 0), fontThickness, cv2.LINE_AA)

        return img



        


    def predict_video(self, video_path, save_dir, save_format = "avi", display = 'custom', verbose = True, **display_args):
        cap = cv2.VideoCapture(video_path)
        # Get video name 
        vid_name = os.path.basename(video_path)


        # Get frame dimensions and print information about input video file
        width  = int(cap.get(3) )  # get `width` 
        height = int(cap.get(4) )  # get `height` 

        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)

        save_name = self.model_name + ' -- ' + vid_name.split('.')[0] + '.' + save_format
        save_file = os.path.join(save_dir, save_name)

        if verbose:
            print("----------------------------")
            print(f"DETECTING OBJECTS IN : {vid_name} : ")
            print(f"RESOLUTION : {width}x{height}")
            print('SAVING TO :' + save_file)

        # define an output VideoWriter  object
        out = cv2.VideoWriter(save_file,
                            cv2.VideoWriter_fourcc(*"MJPG"),
                            30,(width,height))

        # Check if the video is opened correctly
        if not cap.isOpened():
            print("Error opening video stream or file")

        # Read the video frames
        while cap.isOpened():
            ret, frame = cap.read()

            # If the frame was not read successfully, break the loop
            if not ret:
                print("Error reading frame")
                break

            # Run object detection on the frame and calculate FPS
            beg = time.time()
            results = self.predict_img(frame, verbose = False)
            fps = 1 / (time.time() - beg)

            # Display detection results
            if display == 'default':
                frame = self.default_display(**display_args)
            
            elif display == 'custom':
                frame == self.custom_display(colors, **display_args)

            # Display FPS on frame
            frame = cv2.putText(frame,f"FPS : {fps:,.2f}" , 
                                (5,15), cv2.FONT_HERSHEY_COMPLEX, 
                            0.5,  (0,0,255), 1, cv2.LINE_AA)
            

            # append frame to the video file
            out.write(frame)
            
            # the 'q' button is set as the
            # quitting button you may use any
            # desired button of your choice

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # After the loop release the cap 
        cap.release()
        out.release()

    


In [5]:
d = YOLOv8_ObjectDetector()
print(d.labels) 

  0%|          | 0.00/6.23M [00:00<?, ?B/s]

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [6]:
img_results_path = '/content/image_object_detection_results'
vid_results_path = '/content/video_object_detection_results'
test_imgs_path = '/content/Object-tracking-and-counting-using-YOLOV8/test imgs'
test_vids_path = '/content/Object-tracking-and-counting-using-YOLOV8/test vids'

if not os.path.isdir(img_results_path):
    os.makedirs(img_results_path)

if not os.path.isdir(vid_results_path):
    os.makedirs(vid_results_path)

In [7]:
yolo_names = ['yolov8n.pt', 'yolov8m.pt', 'yolov8x.pt']
colors = []
for _ in range(80):
    rand_tuple = (random.randint(50, 255), random.randint(50, 255), random.randint(50, 255))
    colors.append(rand_tuple)

detectors = []
for yolo_name in yolo_names:
    detector = YOLOv8_ObjectDetector(yolo_name, conf = 0.55 )
    detectors.append(detector)

  0%|          | 0.00/49.7M [00:00<?, ?B/s]

  0%|          | 0.00/131M [00:00<?, ?B/s]

In [ ]:
for detector in detectors :
    print(f"Model : {detector.model_name}")
    for img_name in os.listdir(test_imgs_path):
        print(f"Target image : {img_name}")
        print(img_name.split('.')[0])
        print("-------------------------")
        img = cv2.imread(os.path.join(test_imgs_path, img_name))
        results = detector.predict_img(img, verbose = False)
        result_img = detector.custom_display(colors)

        save_dir = os.path.join(img_results_path, detector.model_name)
        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)

        save_name = detector.model_name + ' -- ' + img_name.split('.')[0] + '.jpg'
        save_file = os.path.join(img_results_path, save_dir, save_name) 
        print(f"Saving results to : {save_file}")
        cv2.imwrite(save_file, result_img)


In [13]:
for detector in detectors:
    detector.predict_video(video_path= '/content/Object-tracking-and-counting-using-YOLOV8/test vids/traffic 2.mp4'
, save_dir = vid_results_path, save_format = "avi", display = 'custom')

----------------------------
DETECTING OBJECTS IN : traffic 2.mp4 : 
RESOLUTION : 1280x720
SAVING TO :/content/video_object_detection_results/yolov8n -- traffic 2.avi
Error reading frame
----------------------------
DETECTING OBJECTS IN : traffic 2.mp4 : 
RESOLUTION : 1280x720
SAVING TO :/content/video_object_detection_results/yolov8m -- traffic 2.avi
Error reading frame
----------------------------
DETECTING OBJECTS IN : traffic 2.mp4 : 
RESOLUTION : 1280x720
SAVING TO :/content/video_object_detection_results/yolov8x -- traffic 2.avi
Error reading frame


In [ ]:
!zip -r img_results.zip /content/image_object_detection_results

In [ ]:
!zip -r vid_results.zip /content/video_object_detection_results